Preparing dataset for feeding to NN. Every value is rescaled within  a sample's time window to fit in [0,1] interval.

In [ ]:
import pandas as pd
import numpy as np
import os
import math


rawdata = os.scandir(r'../input/optiver-realized-volatility-prediction/book_train.parquet')
rawdatb = os.scandir(r'../input/optiver-realized-volatility-prediction/trade_train.parquet')

def log_return(list_stock_prices):
    return np.log(list_stock_prices).diff() 

schet=0
datasetB=[]
listID=[]
datasetT=[]

for entry1 in rawdata:
    group=os.scandir(r'../input/optiver-realized-volatility-prediction/book_train.parquet/'+entry1.name)
    foldername=r'../input/optiver-realized-volatility-prediction/book_train.parquet/'+entry1.name
    for entry in group:
        datasetB.append(foldername+'/'+entry.name)
        listID.append(int(entry1.name[9:]))
    schet+=1

for entry1 in rawdatb:
    group=os.scandir(r'../input/optiver-realized-volatility-prediction/trade_train.parquet/'+entry1.name)
    foldername=r'../input/optiver-realized-volatility-prediction/trade_train.parquet/'+entry1.name
    for entry in group:
        datasetT.append(foldername+'/'+entry.name)
    schet+=1    

dataset=[]

for i in range(len(listID)):
    stringI=[]
    stringI.append(listID[i])
    stringI.append(datasetB[i])
    stringI.append(datasetT[i])
    dataset.append(stringI)
    
dataset.sort()

dataset=pd.DataFrame(dataset)

n=pd.read_parquet(dataset[1][2])


x0=6
delta=int(600/x0)

     
        
labels=pd.read_csv(r'../input/optiver-realized-volatility-prediction/train.csv')        
          


bigdata=np.zeros((len(labels),(x0)*6+2))


sIDP=-50

for i in range(len(labels)):
    sID=labels.iat[i,0]
    tID=labels.iat[i,1]
    target=labels.iat[i,2]
    if sIDP!=sID:
        book=pd.read_parquet(dataset.loc[dataset[0]==sID,[1]].iat[0,0])
        trade=pd.read_parquet(dataset.loc[dataset[0]==sID,[2]].iat[0,0])
    sIDP=sID  
    bigdata[i,(x0)*6+1]=target
    b=book.loc[book['time_id']==tID]
    t=trade.loc[trade['time_id']==tID]
    b.loc[:,'wap']=log_return((b.loc[:,'bid_price1']*b.loc[:,'ask_size1']+b.loc[:,'ask_price1']*b.loc[:,'bid_size1'])/(b.loc[:,'bid_size1']+b.loc[:,'ask_size1']))
    b.loc[b['wap'].isnull()==1,'wap']=0
    bmax=b.loc[:,'seconds_in_bucket'].max()
    bmin=b.loc[:,'seconds_in_bucket'].min()
    bdelta=(bmax-bmin)/599
    b.loc[:,'seconds_in_bucket']=(b.loc[:,'seconds_in_bucket']-bmin)/bdelta
    t.loc[:,'seconds_in_bucket']=(t.loc[:,'seconds_in_bucket']-bmin)/bdelta
    b.loc[:,'seconds_in_bucket']=b.loc[:,'seconds_in_bucket']//delta
    t.loc[:,'seconds_in_bucket']=t.loc[:,'seconds_in_bucket']//delta
    totalo1=np.sum(b.loc[:,'ask_size1' ])
    totalo2=np.sum(b.loc[:,'bid_size1'])
    totalt=np.sum(t.loc[:,'size'])
    totalo3=np.sum((b.loc[:, 'ask_price1']-b.loc[:, 'bid_price1']))
    #totalo4=np.sum((b.loc[:, 'ask_price1']-b.loc[:, 'ask_price2']))

    for k in range(x0):
            step=b.loc[b['seconds_in_bucket']<=k]
            f1=np.sqrt(np.sum(np.square(step.loc[:,'wap'])))
            if math.isnan(f1):
                f1=0
            bigdata[i,k]=f1
            f2=np.sum(step.loc[:, 'ask_size1'])/totalo1
            if math.isnan(f2):
                f2=0
            bigdata[i,k+1*x0]=f2
            step1=t.loc[t['seconds_in_bucket']<=k]
            f3=np.sum(step.loc[:, 'bid_size1'])/totalo2
            if math.isnan(f3):
                f3=0
            bigdata[i,k+2*x0]=f3
            f4=np.sum(step.loc[:, 'ask_price1']-step.loc[:, 'bid_price1'])/totalo3
            if math.isnan(f4):
                f4=0
            bigdata[i,k+3*x0]=f4
            f5=np.sum(step1.loc[:, 'size'])/totalt
            if math.isnan(f5):
                f5=0
            bigdata[i,k+4*x0]=f5
            f6=np.sum(step.loc[:, 'ask_price1']-step.loc[:, 'ask_price2'])/totalo3#wastotalo4
            if math.isnan(f6):
                f6=0
            bigdata[i,k+5*x0]=f6
    bigdata[i,6*x0]=sIDP         

Neural network is trained in a few steps. Each step aims to train separate layers of the network, i.e. the first step trains convolutional part, the second trains the LSTM layer. The experiments has shown, that training with MeanAbsolutePercentageError instead of the  loss function proposed by competition description shows slightly better reuslts in some cases and in some cases the results are identical.

In [ ]:
from  tensorflow import keras
from tensorflow.keras import layers



x=6

dataset=bigdata
np.random.shuffle(dataset)
f1,f2,f3,f4,f5,f6,sID,trg=np.hsplit(dataset, [x, 2*x, 3*x, 4*x, 5*x, 6*x, 6*x+1])
sidB=np.zeros((len(dataset),127))
for i in range(len(sidB)):
    pos=int(sID[i])
    sidB[i,pos]=1


i1=keras.Input(shape=(x,1), name='i1')
i2=keras.Input(shape=(x,1), name='i2')
i3=keras.Input(shape=(x,1), name='i3')
i4=keras.Input(shape=(x,1), name='i4')
i5=keras.Input(shape=(x,1), name='i5')
i6=keras.Input(shape=(x,1), name='i6')

i7=keras.Input(shape=(127,1), name='i7')




sid=layers.Conv1D(
    3,
    127,
    strides=1,
    padding="valid",
    activation='sigmoid',
    use_bias=True,
    kernel_initializer="zeros",
    bias_initializer="zeros",
    kernel_regularizer=None,
    bias_regularizer=None,
    activity_regularizer=None,
    kernel_constraint=None,
    bias_constraint=None,
    )(i7)



sid=layers.Flatten()(sid)

pt=layers.concatenate([i1,i2,i3,i4,i5,i6],axis=2)

pt=layers.LSTM(
    40,
    activation="tanh",
    recurrent_activation="sigmoid",
    use_bias=True,
    kernel_initializer="glorot_uniform",
    recurrent_initializer="orthogonal",
    bias_initializer="zeros",
    unit_forget_bias=True,
    kernel_regularizer=None,
    recurrent_regularizer=None,
    bias_regularizer=None,
    activity_regularizer=None,
    kernel_constraint=None,
    recurrent_constraint=None,
    bias_constraint=None,
    dropout=0.4, 
    recurrent_dropout=0.0,
    return_sequences=False,
    return_state=False,
    go_backwards=True,
    stateful=False,
    time_major=False,
    unroll=False,)(pt)


pt=layers.concatenate([pt,sid],axis=1)

xx=layers.Dense(10,activation="relu", use_bias=True)(pt)


catN=layers.Dense(1,activation="sigmoid", use_bias=True, name='output')(xx)

callback = keras.callbacks.EarlyStopping(monitor='val_loss',  patience=10, baseline=100, mode='auto', restore_best_weights=True)
callback1 = keras.callbacks.EarlyStopping(monitor='val_loss',  patience=10, baseline=100, mode='auto', restore_best_weights=True)

model = keras.Model(inputs=[i1,i2,i3,i4,i5,i6,i7], outputs=catN)



opt = keras.optimizers.Adam(learning_rate=0.003)
model.layers[8].trainable = False
model.compile(loss='MeanAbsolutePercentageError', 
              optimizer=opt,  metrics=[keras.metrics.MeanAbsolutePercentageError()])
model.summary()
for l in model.layers:
    print(l.name, l.trainable)
fitting=model.fit({'i1':f1,'i2':f2, 'i3':f3, 'i4':f4, 'i5':f5, 'i6':f6, 'i7':sidB},{'output':trg}, 
          epochs=100, validation_split=0.5, callbacks=[callback]) 
weights1 = model.get_weights()



np.random.shuffle(dataset)
f1,f2,f3,f4,f5,f6,sID,trg=np.hsplit(dataset, [x, 2*x, 3*x, 4*x, 5*x, 6*x, 6*x+1])
sidB=np.zeros((len(dataset),127))
for i in range(len(sidB)):
    pos=int(sID[i])
    sidB[i,pos]=1

model1=keras.Model(inputs=[i1,i2,i3,i4,i5,i6,i7], outputs=catN)

model1.set_weights(weights1)
model1.layers[9].trainable = False
model1.layers[8].trainable = True


model1.compile(loss='MeanAbsolutePercentageError', 
              optimizer=opt,  metrics=[keras.metrics.MeanAbsolutePercentageError()])
#for l in model.layers:
#    print(l.name, l.trainable)
#model.summary()
fitting=model1.fit({'i1':f1,'i2':f2, 'i3':f3, 'i4':f4, 'i5':f5, 'i6':f6, 'i7':sidB},{'output':trg}, 
          epochs=30, validation_split=0.5, callbacks=[callback1]) 


Preparing test dataset:

In [ ]:

rawdataT = os.scandir(r'../input/optiver-realized-volatility-prediction/book_test.parquet')
rawdatbT = os.scandir(r'../input/optiver-realized-volatility-prediction/trade_test.parquet')

schet=0
datasetB=[]
listID=[]
datasetT=[]

for entry1 in rawdataT:
    group=os.scandir(r'../input/optiver-realized-volatility-prediction/book_test.parquet/'+entry1.name)
    foldername=r'../input/optiver-realized-volatility-prediction/book_test.parquet/'+entry1.name
    for entry in group:
        datasetB.append(foldername+'/'+entry.name)
        listID.append(int(entry1.name[9:]))
    schet+=1

for entry1 in rawdatbT:
    group=os.scandir(r'../input/optiver-realized-volatility-prediction/trade_test.parquet/'+entry1.name)
    foldername=r'../input/optiver-realized-volatility-prediction/trade_test.parquet/'+entry1.name
    for entry in group:
        datasetT.append(foldername+'/'+entry.name)
    schet+=1    

dataset=[]

for i in range(len(listID)):
    stringI=[]
    stringI.append(listID[i])
    stringI.append(datasetB[i])
    stringI.append(datasetT[i])
    dataset.append(stringI)
    
dataset.sort()


submission=[]
labels=[]

for i in range(len(dataset)):
    n=pd.read_parquet(dataset[i][1])
    stockID=dataset[i][0]
    timeID=n['time_id'].unique()
    for j in range(len(timeID)):
        submission.append(str(stockID)+'-'+str(timeID[j]))
        labels.append([stockID,timeID[j]])
        


x0=6
delta=int(600/x0)


bigdata=np.zeros((len(submission),(x0)*6+1))

sIDP=-50
dataset=pd.DataFrame(dataset)
for i in range(len(labels)):
    sID=labels[i][0]
    tID=labels[i][1]
    if sIDP!=sID:
        book=pd.read_parquet(dataset.loc[dataset[0]==sID,[1]].iat[0,0])
        trade=pd.read_parquet(dataset.loc[dataset[0]==sID,[2]].iat[0,0])
    sIDP=sID  
    b=book.loc[book['time_id']==tID]
    t=trade.loc[trade['time_id']==tID]
    b.loc[:,'wap']=log_return((b.loc[:,'bid_price1']*b.loc[:,'ask_size1']+b.loc[:,'ask_price1']*b.loc[:,'bid_size1'])/(b.loc[:,'bid_size1']+b.loc[:,'ask_size1']))
    b.loc[b['wap'].isnull()==1,'wap']=0
    bmax=b.loc[:,'seconds_in_bucket'].max()
    bmin=b.loc[:,'seconds_in_bucket'].min()
    bdelta=(bmax-bmin)/599
    b.loc[:,'seconds_in_bucket']=(b.loc[:,'seconds_in_bucket']-bmin)/bdelta
    t.loc[:,'seconds_in_bucket']=(t.loc[:,'seconds_in_bucket']-bmin)/bdelta
    b.loc[:,'seconds_in_bucket']=b.loc[:,'seconds_in_bucket']//delta
    t.loc[:,'seconds_in_bucket']=t.loc[:,'seconds_in_bucket']//delta
    totalo1=np.sum(b.loc[:,'ask_size1' ])
    totalo2=np.sum(b.loc[:,'bid_size1'])
    totalt=np.sum(t.loc[:,'size'])
    totalo3=np.sum((b.loc[:, 'ask_price1']-b.loc[:, 'bid_price1']))
    totalo4=np.sum((b.loc[:, 'ask_price1']-b.loc[:, 'ask_price2']))

    for k in range(x0):
            step=b.loc[b['seconds_in_bucket']<=k]
            f1=np.sqrt(np.sum(np.square(step.loc[:,'wap'])))
            if math.isnan(f1):
                f1=0
            bigdata[i,k]=f1
            f2=np.sum(step.loc[:, 'ask_size1'])/totalo1
            if math.isnan(f2):
                f2=0
            bigdata[i,k+1*x0]=f2
            step1=t.loc[t['seconds_in_bucket']<=k]
            f3=np.sum(step.loc[:, 'bid_size1'])/totalo2
            if math.isnan(f3):
                f3=0
            bigdata[i,k+2*x0]=f3
            f4=np.sum(step.loc[:, 'ask_price1']-step.loc[:, 'bid_price1'])/totalo3
            if math.isnan(f4):
                f4=0
            bigdata[i,k+3*x0]=f4
            f5=np.sum(step1.loc[:, 'size'])/totalt
            if math.isnan(f5):
                f5=0
            bigdata[i,k+4*x0]=f5
            f6=np.sum(step.loc[:, 'ask_price1']-step.loc[:, 'ask_price2'])/totalo4
            if math.isnan(f6):
                f6=0
            bigdata[i,k+5*x0]=f6
    bigdata[i,6*x0]=sIDP  

In [ ]:
Submitting results:

In [ ]:
dataset=bigdata

f1,f2,f3,f4,f5,f6,sID=np.hsplit(dataset, [x0, 2*x0, 3*x0, 4*x0, 5*x0, 6*x0])
sidB=np.zeros((len(dataset),127))
for i in range(len(sidB)):
    pos=int(sID[i])
    sidB[i,pos]=1
    
nn=model1.predict(
    x={'i1':f1,'i2':f2, 'i3':f3, 'i4':f4, 'i5':f5, 'i6':f6, 'i7':sidB}, batch_size=None, verbose=0, steps=None,
    callbacks=None)

n=nn.reshape((len(nn)))
pred=n.tolist()

lst1 = submission
lst2 = pred

itog = pd.DataFrame(
    {'row_id': lst1,
     'target': lst2
     
    })


itog.to_csv('submission.csv',index=False)
    